## Welcome to the EURO-NMD Diagnosis Delay Analytics Page (KPI 000007)

The KPI_000007 Data Service calculates the offset between symptom onset and diagnosis for each diagnosis in a registry, grouped and averaged by year  (e.g. Duchenne, 1990, 379   means that for Duchenne diagnoses made in 1990, the average symptom-onset-to-diagnosis difference for all patients diagnosed in 1990 was 379 days)

Please run the first cell to set-up the analytics environment

In the subsequent cells, we have pre-filled a basic analytics, to show you how to access and manipulate the data that was retrieved from the FLAIR-GG Virtual Platform.  

Fill in the "key = 'XXXXXXX' with the secret key for your federated exploration output, and then... go!

In [ ]:
import pyodide_kernel

%piplite install ipywidgets==8.1.1
%piplite install ipyevents==2.0.2
%piplite install ipympl==0.9.4
%piplite install ipycanvas==0.13.2
%piplite install ipyleaflet==0.18.2
%piplite install plotly==5.24.0
%piplite install bqplot==0.12.45
%pip install altair
%pip install pandas
%pip install requests

import altair as alt
import pandas as pd
import pyodide_http
import ssl
import json
import requests
import urllib3
import urllib.parse
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print("DONE!  Proceed to next cell and replace the XXXXX with your secret key")

In [ ]:
import io
import array
import sys

key = "XXXXXXXXX"   # your secret key from the FLAIR-GG Virtual Platform


url = "https://bgv.cbgp.upm.es/DAV/home/LDP/FLAIR/{}".format(key)


if key == "XXXXXXXXX":
    sys.exit("you didn't change the key!")

# ORDO,yearOfDiagnosis,avgoffset
# http://www.orpha.net/ORDO/Orphanet_98896,1986,9949
# http://www.orpha.net/ORDO/Orphanet_98896,1996,6469
# http://www.orpha.net/ORDO/Orphanet_98896,1999,5250

response = requests.get(url)
response = json.loads(response.content)
#print(response)

data = {}
for provider in response.keys():
    print("Provider: {}".format(provider))
    alllines = response[provider]
    data[provider] = []

    lines = alllines.splitlines()  # this is the CSV
    lines.pop(0)  # get rid of header
    # print(data)
    for line in iter(lines):
        [diag, year, offset] = line.split(",")
        diag = re.sub(r"^.*[/#]", "", diag)
        data[provider].append([diag, year, offset])

# print(data)
print("DONE")

In [ ]:
data_list = []
for provider, offsets in data.items():
    for diag, year, offset in offsets:
        data_list.append({"Provider": provider, "Diagnosis": diag, "Year": year, "Offset(days)": offset})

# Create a DataFrame
df = pd.DataFrame(data_list)

In [ ]:
# Horizontal Bar Chart
horizontal_bar_chart = alt.Chart(df).mark_bar().encode(
    y=alt.Y('Year:O', title='Year'),  # Swap x and y for horizontal bars
    x=alt.X('Offset(days):Q', title='Offset (Days)'),
    color=alt.Color('Provider:N', title='Provider'),
    tooltip=['Provider', 'Diagnosis', 'Year', 'Offset(days)'],
    row=alt.Row('Diagnosis:N', title='Diagnosis')  # Separate rows for each diagnosis
).properties(
    title='Offset Days by Year and Provider',
    width=600,
    height=150
).configure_axisX(
    labelAngle=0  # Ensure x-axis labels are horizontal
)

horizontal_bar_chart

In [ ]:
# Heatmap
heatmap = alt.Chart(df).mark_rect().encode(
    x=alt.X('Year:O', title='Year'),
    y=alt.Y('Provider:N', title='Provider'),
    color=alt.Color('Offset(days):Q', scale=alt.Scale(scheme='inferno'), title='Offset (Days)'),  # Using 'inferno' color scheme
    tooltip=['Provider', 'Diagnosis', 'Year', 'Offset(days)']
).properties(
    title='Offset Days Heatmap',
    width=600,
    height=200
)

heatmap



# CLEAR NOTEBOOK CACHE

The code below will erase the cache and force this Jupyter Notebook to be reloaded from-scratch when you click the "reload" button.

You will lose all work and all code if you do this!  You have been warned :-)


In [ ]:
from IPython.display import display, HTML
display(HTML("""
<button type="button" id="button_for_indexeddb">Clear JupyterLite local storage</button>
<script>
window.button_for_indexeddb.onclick = function(e) {
    window.indexedDB.open('JupyterLite Storage').onsuccess = function(e) {
        // There are also other tables that I'm not clearing:
        // "counters", "settings", "local-storage-detect-blob-support"
        let tables = ["checkpoints", "files"];

        let db = e.target.result;
        let t = db.transaction(tables, "readwrite");

        function clearTable(tablename) {
            let st = t.objectStore(tablename);
            st.count().onsuccess = function(e) {
                console.log("Deleting " + e.target.result + " entries from " + tablename + "...");
                st.clear().onsuccess = function(e) {
                    console.log(tablename + " is cleared!");
                }
            }
        }

        for (let tablename of tables) {
            clearTable(tablename);
        }
    }
};
</script>
"""))

